In [1]:
import csv
from pathlib import Path

import pandas as pd

In [2]:
model = "wic6"
wic_path = f"../input/wic/dwug_es_cleaned/models/{model}/train"
wic_path_input = "../input/wic/dwug_es_cleaned"
path_to_gold_data_dwug_en = "../test_data_es.csv"
path_to_annotated_data_dwug_en = "../dwug_es_cleaned/data"

In [3]:
p = next(Path(wic_path_input).glob("*.data"))
data_input = pd.read_json(p)
display(data_input)

,id,lemma,pos,sentence1,sentence2,start1,end1,start2,end2
0,dev.elocuente.0,elocuente,NOUN,"Hablaba de todo y de nada. No concretaba, y su...","Entonces, el compañero de su padre, conociendo...",48,58,428,437
1,dev.elocuente.1,elocuente,NOUN,"Hablaba de todo y de nada. No concretaba, y su...",¿Por qué no había observado más temprano esa f...,48,58,175,185
2,dev.elocuente.2,elocuente,NOUN,"Hablaba de todo y de nada. No concretaba, y su...",¡Se comprende que su peregrino cacumen hallara...,48,58,185,194
3,dev.elocuente.3,elocuente,NOUN,"Hablaba de todo y de nada. No concretaba, y su...","Saliendo a la galería miró al patio, y lo prim...",48,58,366,376
4,dev.elocuente.4,elocuente,NOUN,"Hablaba de todo y de nada. No concretaba, y su...",Recogí del suelo una pequeña cinta y unos peda...,48,58,306,316
...,...,...,...,...,...,...,...,...,...
46795,dev.ataque.77995,ataque,NOUN,"Sí, vieron a la madre que fue asesinada mientr...",Vd. sí que tiene la cabeza llena de arroyos y ...,151,158,129,135
46796,dev.ataque.77996,ataque,NOUN,"Sí, vieron a la madre que fue asesinada mientr...","En fin, que si no acierta a pasar el coronel G...",151,158,244,250
46797,dev.ataque.77997,ataque,NOUN,"Sí, vieron a la madre que fue asesinada mientr...","Pero nosotros teníamos, para sojuzgar a los tr...",151,158,244,250
46798,dev.ataque.77998,ataque,NOUN,"Sí, vieron a la madre que fue asesinada mientr...","Con doble juego de fusiles, Sabas y un viejo d...",151,158,219,225


In [4]:
mask = data_input["sentence1"] == data_input["sentence2"]
data_input = data_input[~mask]
print(data_input.shape)

(46773, 9)


In [5]:
mask = data_input.duplicated(subset=["sentence1", "sentence2"], keep=False)
data_input = data_input[~mask]
print(data_input.shape)

(45110, 9)


In [6]:
data_to_be_concatenated = [pd.read_json(data) for data in Path(wic_path).rglob("*.scores")]
data_scores = pd.concat(data_to_be_concatenated, ignore_index=True)
display(data_scores)

,id,score
0,dev.recordar.26520,"[0.5008608, 0.50287384]"
1,dev.recordar.26521,"[0.5035621, 0.50440055]"
2,dev.recordar.26522,"[0.50664675, 0.5076339]"
3,dev.recordar.26523,"[0.5083486, 0.514088]"
4,dev.recordar.26524,"[0.49795112, 0.49891034]"
...,...,...
46795,dev.distribuir.4675,"[0.5079415, 0.5043334]"
46796,dev.distribuir.4676,"[0.48826396, 0.48542243]"
46797,dev.distribuir.4677,"[0.52240103, 0.523398]"
46798,dev.distribuir.4678,"[0.4869759, 0.48612496]"


In [7]:
gold_data_dwug_en = pd.read_csv(path_to_gold_data_dwug_en, sep="\t")
display(gold_data_dwug_en)

,word,change_binary,change_binary_gain,change_binary_loss,change_graded,COMPARE
0,actitud,1,1,1,0.924661,2.242958
1,ataque,1,0,1,0.481434,2.575221
2,atrás,0,0,0,0.159570,2.947500
3,ausencia,0,0,0,0.000000,3.389163
4,avance,0,0,0,0.590113,2.106884
5,banco,1,1,1,0.924661,1.267544
6,canal,1,1,1,1.000000,1.613462
7,capital,0,0,0,0.446772,2.157088
8,cobrar,1,0,1,0.217259,2.469697
9,colaborar,1,0,1,0.557923,2.599548


In [8]:
target_words_dwug_en = gold_data_dwug_en["word"].unique().tolist()
target_words_dwug_en

['actitud',
 'ataque',
 'atrás',
 'ausencia',
 'avance',
 'banco',
 'canal',
 'capital',
 'cobrar',
 'colaborar',
 'cólera',
 'compasión',
 'copiar',
 'corriente',
 'declinar',
 'demá',
 'diligencia',
 'disco',
 'distribuir',
 'educado',
 'elocuente',
 'encargado',
 'enterar',
 'especulación',
 'fallar',
 'fallecimiento',
 'historia',
 'historiador',
 'impulso',
 'indicativo',
 'juguete',
 'maduro',
 'maravilloso',
 'marco',
 'matiz',
 'médula',
 'metal',
 'metro',
 'modificado',
 'nombrar',
 'pendiente',
 'pila',
 'planta',
 'prima',
 'propiamente',
 'próximo',
 'recomendar',
 'recordar',
 'retroceder',
 'satélite',
 'socialista',
 'solicitud',
 'susceptible',
 'tarea',
 'trato',
 'tropical',
 'variedad',
 'viernes',
 'visita',
 'vuestro']

In [9]:
data_input_expanded = data_input.merge(data_scores, on="id", how="inner")
data_input_expanded.drop(columns=["pos"], inplace=True)
display(data_input_expanded)

mask = data_input_expanded["lemma"].isin(target_words_dwug_en)
data_input_expanded_filtered = data_input_expanded[mask]
data_input_expanded_filtered = data_input_expanded_filtered.drop(columns=["start1", "end1", "start2", "end2"])
display(data_input_expanded_filtered)
print(len(data_input_expanded_filtered["lemma"].unique().tolist()))


,id,lemma,sentence1,sentence2,start1,end1,start2,end2,score
0,dev.elocuente.0,elocuente,"Hablaba de todo y de nada. No concretaba, y su...","Entonces, el compañero de su padre, conociendo...",48,58,428,437,"[0.5115616, 0.5132249]"
1,dev.elocuente.1,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",¿Por qué no había observado más temprano esa f...,48,58,175,185,"[0.5080961, 0.505037]"
2,dev.elocuente.2,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",¡Se comprende que su peregrino cacumen hallara...,48,58,185,194,"[0.5031368, 0.4998717]"
3,dev.elocuente.3,elocuente,"Hablaba de todo y de nada. No concretaba, y su...","Saliendo a la galería miró al patio, y lo prim...",48,58,366,376,"[0.49114358, 0.4928302]"
4,dev.elocuente.4,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",Recogí del suelo una pequeña cinta y unos peda...,48,58,306,316,"[0.5015791, 0.49905273]"
...,...,...,...,...,...,...,...,...,...
45105,dev.ataque.77995,ataque,"Sí, vieron a la madre que fue asesinada mientr...",Vd. sí que tiene la cabeza llena de arroyos y ...,151,158,129,135,"[0.5074098, 0.5069284]"
45106,dev.ataque.77996,ataque,"Sí, vieron a la madre que fue asesinada mientr...","En fin, que si no acierta a pasar el coronel G...",151,158,244,250,"[0.48286763, 0.48212156]"
45107,dev.ataque.77997,ataque,"Sí, vieron a la madre que fue asesinada mientr...","Pero nosotros teníamos, para sojuzgar a los tr...",151,158,244,250,"[0.51103634, 0.5096609]"
45108,dev.ataque.77998,ataque,"Sí, vieron a la madre que fue asesinada mientr...","Con doble juego de fusiles, Sabas y un viejo d...",151,158,219,225,"[0.50411046, 0.51117384]"


,id,lemma,sentence1,sentence2,score
0,dev.elocuente.0,elocuente,"Hablaba de todo y de nada. No concretaba, y su...","Entonces, el compañero de su padre, conociendo...","[0.5115616, 0.5132249]"
1,dev.elocuente.1,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",¿Por qué no había observado más temprano esa f...,"[0.5080961, 0.505037]"
2,dev.elocuente.2,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",¡Se comprende que su peregrino cacumen hallara...,"[0.5031368, 0.4998717]"
3,dev.elocuente.3,elocuente,"Hablaba de todo y de nada. No concretaba, y su...","Saliendo a la galería miró al patio, y lo prim...","[0.49114358, 0.4928302]"
4,dev.elocuente.4,elocuente,"Hablaba de todo y de nada. No concretaba, y su...",Recogí del suelo una pequeña cinta y unos peda...,"[0.5015791, 0.49905273]"
...,...,...,...,...,...
45105,dev.ataque.77995,ataque,"Sí, vieron a la madre que fue asesinada mientr...",Vd. sí que tiene la cabeza llena de arroyos y ...,"[0.5074098, 0.5069284]"
45106,dev.ataque.77996,ataque,"Sí, vieron a la madre que fue asesinada mientr...","En fin, que si no acierta a pasar el coronel G...","[0.48286763, 0.48212156]"
45107,dev.ataque.77997,ataque,"Sí, vieron a la madre que fue asesinada mientr...","Pero nosotros teníamos, para sojuzgar a los tr...","[0.51103634, 0.5096609]"
45108,dev.ataque.77998,ataque,"Sí, vieron a la madre que fue asesinada mientr...","Con doble juego de fusiles, Sabas y un viejo d...","[0.50411046, 0.51117384]"


60


In [10]:
mask = data_input_expanded_filtered["sentence1"] == data_input_expanded_filtered["sentence2"]
display(data_input_expanded_filtered[mask].shape)

(0, 5)

In [11]:
annotated_data_to_be_concatenated_dwug_en = [
    pd.read_csv(
        p,
        # engine="python",
        # on_bad_lines="warn",
        sep="\t",
        encoding="utf-8",
        quoting=csv.QUOTE_MINIMAL,
        quotechar='"',
    )
    for p in Path(path_to_annotated_data_dwug_en).rglob("*uses.csv")
]
annotated_data_dwug_en = pd.concat(
    annotated_data_to_be_concatenated_dwug_en, ignore_index=True
)
display(annotated_data_dwug_en)

mask = annotated_data_dwug_en["lemma"].isin(target_words_dwug_en)
annotated_data_dwug_en_filtered = annotated_data_dwug_en[mask]
annotated_data_dwug_en_filtered = annotated_data_dwug_en_filtered[
    ["lemma", "grouping", "identifier", "context"]
]
# display(annotated_data_dwug_es_filtered)
annotated_data_dwug_en_filtered = annotated_data_dwug_en_filtered[
    ~annotated_data_dwug_en_filtered.duplicated(subset="context", keep=False)
]
display(annotated_data_dwug_en_filtered)

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence
0,abundar,VERB,1858,1,old_corpus_spanish-17804-364,,"Fueron concluidos en 1570. Hay, por fin, tres ...",178:185,27:186
1,abundar,VERB,1858,1,old_corpus_spanish-17808-437,,"Ya que hemos hablado de los atriles, permitase...",215:222,110:419
2,abundar,VERB,1858,1,old_corpus_spanish-312321-337,,Cosas muy buenas han legislado y dispuesto nue...,165:172,98:212
3,abundar,VERB,1858,1,old_corpus_spanish-78063-539,,El techo del cuerpo principal estaba formado c...,389:397,172:425
4,abundar,VERB,1858,1,old_corpus_spanish-354246-657,,Veíase metido en el charco de las ranas pidien...,324:333,187:372
...,...,...,...,...,...,...,...,...,...
3995,especulación,NOUN,1858,1,old_corpus_spanish-413722-690,,"Solamente te diré, que pues la moda quiere que...",339:351,260:376
3996,especulación,NOUN,1858,1,old_corpus_spanish-405874-450,,María guardaba siempre en su poder porción cum...,219:231,216:316
3997,especulación,NOUN,1858,1,old_corpus_spanish-34941-198,,Hasta entonces el puerto de Cádiz era el único...,93:105,68:186
3998,especulación,NOUN,1858,1,old_corpus_spanish-181679-897,,Se ha declamado mucho contra el positivismo de...,442:454,378:619


,lemma,grouping,identifier,context
40,fallecimiento,1,old_corpus_spanish-658493-483,"Unidos ambos en estrecho abrazo, diéronse palm..."
41,fallecimiento,1,old_corpus_spanish-172270-847,Toda esta paz la trastornó la muerte de mi mad...
43,fallecimiento,1,old_corpus_spanish-196412-632,"Una junta patronal, formada por Juan Casado, d..."
44,fallecimiento,1,old_corpus_spanish-110390-862,Su pronóstico recae sobre el número total de l...
46,fallecimiento,1,old_corpus_spanish-18713-717,"Atribuyéronla, unos á la peste que entonces an..."
...,...,...,...,...
3994,especulación,1,old_corpus_spanish-30720-58,"Dispénseme. Yo no bebo ni a la salud del Papa,..."
3996,especulación,1,old_corpus_spanish-405874-450,María guardaba siempre en su poder porción cum...
3997,especulación,1,old_corpus_spanish-34941-198,Hasta entonces el puerto de Cádiz era el único...
3998,especulación,1,old_corpus_spanish-181679-897,Se ha declamado mucho contra el positivismo de...


In [12]:
merged1 = data_input_expanded_filtered.merge(
    annotated_data_dwug_en_filtered,
    left_on=["lemma", "sentence1"],
    right_on=["lemma", "context"],
    how="inner",
).rename(columns={"identifier": "identifier1"})


merged2 = merged1.merge(
    annotated_data_dwug_en_filtered,
    left_on=["lemma", "sentence2"],
    right_on=["lemma", "context"],
    how="inner",
).rename(columns={"identifier": "identifier2"})
print(merged2.shape)

result = merged2[
    [
        "id",
        "lemma",
        "sentence1",
        "sentence2",
        "score",
        "identifier1",
        "identifier2",
        "grouping_x",
    ]
]
result["score"] = result["score"].apply(lambda x: sum(float(e) for e in x) / len(x))
result.rename(columns={"grouping_x": "grouping", "lemma": "word"}, inplace=True)
display(result)

(44144, 11)


/tmp/ipykernel_1848993/3675419771.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["score"] = result["score"].apply(lambda x: sum(float(e) for e in x) / len(x))
/home/fzamora/miniconda3/envs/ex/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,id,word,sentence1,sentence2,score,identifier1,identifier2,grouping
0,dev.elocuente.0,elocuente,"Hablaba de todo y de nada. No concretaba, y su...","Entonces, el compañero de su padre, conociendo...",0.512393,old_corpus_spanish-420995-106,old_corpus_spanish-633124-866,1
1,dev.elocuente.16,elocuente,"Cuando se cumplieron estas condiciones, el nov...","Entonces, el compañero de su padre, conociendo...",0.511906,modern_corpus_spanish-649358-375,old_corpus_spanish-633124-866,2
2,dev.elocuente.42,elocuente,Desde la Gran Depresión y la Segunda Guerra Mu...,"Entonces, el compañero de su padre, conociendo...",0.504753,modern_corpus_spanish-689869-617,old_corpus_spanish-633124-866,2
3,dev.elocuente.75,elocuente,"Además, la malnutrición intrauterina puede aum...","Entonces, el compañero de su padre, conociendo...",0.491714,modern_corpus_spanish-533049-389,old_corpus_spanish-633124-866,2
4,dev.elocuente.114,elocuente,"Señor Presidente, me permito felicitar a la Co...","Entonces, el compañero de su padre, conociendo...",0.494229,modern_corpus_spanish-131937-567,old_corpus_spanish-633124-866,2
...,...,...,...,...,...,...,...,...
44139,dev.ataque.77781,ataque,Estos actos son los que ponen en peligro la pa...,¿No fue una provocación que Netanyahu en meses...,0.486865,modern_corpus_spanish-360353-1017,modern_corpus_spanish-549298-913,2
44140,dev.ataque.77978,ataque,"Sí, vieron a la madre que fue asesinada mientr...",¿No fue una provocación que Netanyahu en meses...,0.480417,modern_corpus_spanish-138845-309,modern_corpus_spanish-549298-913,2
44141,dev.ataque.77350,ataque,Sr. Insanally (Guyana) (habla en inglés): Aunq...,Estos actos son los que ponen en peligro la pa...,0.513774,modern_corpus_spanish-308380-209,modern_corpus_spanish-360353-1017,2
44142,dev.ataque.77982,ataque,"Sí, vieron a la madre que fue asesinada mientr...",Estos actos son los que ponen en peligro la pa...,0.505658,modern_corpus_spanish-138845-309,modern_corpus_spanish-360353-1017,2


In [13]:
rows_with_nan = result[result.isna().any(axis=1)]  # or df[df.isnull().any(axis=1)]

print(rows_with_nan)

Empty DataFrame
Columns: [id, word, sentence1, sentence2, score, identifier1, identifier2, grouping]
Index: []


In [14]:
result.to_csv(f"{model}.scores", index=False)

In [15]:
# for tw in target_words_dwug_en:
#     mask = result["word"] == tw
#     df = result[mask]

#     print(tw, df.shape[0])
#     assert df.shape[0] <= 780
    

In [16]:
A = pd.DataFrame({"sentence1": ["bla bla A", "bla bla B", "bla bla C"], "sentence2": ["bla bla C", "bla bla A", "bla bla B"]})
B = pd.DataFrame({"context": ["bla bla B", "bla bla C", "bla bla A"], "identifier": ["b1", "c1", "a1"]})
display(A)
print()
display(B)

,sentence1,sentence2
0,bla bla A,bla bla C
1,bla bla B,bla bla A
2,bla bla C,bla bla B


,context,identifier
0,bla bla B,b1
1,bla bla C,c1
2,bla bla A,a1


In [17]:
merged1 = A.merge(B, left_on="sentence1", right_on="context", how="left")
display(merged1)

,sentence1,sentence2,context,identifier
0,bla bla A,bla bla C,bla bla A,a1
1,bla bla B,bla bla A,bla bla B,b1
2,bla bla C,bla bla B,bla bla C,c1
